libraries

In [2]:
%run import_modules.py  
%matplotlib inline

# For DNN modeling
import tensorflow as tf

# Tensorflow warning off
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.random.set_seed(2020)
import keras
from keras import backend as K
from keras.layers import * #Input, Dense
from keras.models import * #Model
from keras.optimizers import *
from keras.initializers import *
from keras.regularizers import *
from keras.utils.np_utils import *
from keras.utils.vis_utils import * #model_to_dot

Using TensorFlow backend.


load data

In [18]:
df_train = pd.read_csv('X_train.csv', encoding='cp949')
df_test = pd.read_csv('X_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949').gender
IDtest = df_test.cust_id.unique()

# 중분류 구매건수

In [3]:
df_train = pd.read_csv('X_train.csv', encoding='cp949')
df_test = pd.read_csv('X_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949').gender
IDtest = df_test.cust_id.unique()


level = 'gds_grp_nm'

train_test = pd.pivot_table(pd.concat([df_train, df_test]), index='cust_id', columns=level, values='amount',
                            aggfunc=lambda x: len(x), fill_value=0).reset_index()


# 이상치(outlier)를 제거한다.
train_test.iloc[:,1:] = train_test.iloc[:,1:].apply(lambda x: x.clip(x.quantile(.05), x.quantile(.95)), axis=0)

# 왼쪽으로 치우진 분포를 정규분포로 바꾸기 위해 로그 변환을 수행한다. -> 0.769
train_test.iloc[:,1:] = np.log1p(train_test.iloc[:,1:])

#from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler()
train_test.iloc[:, 1:] = mmscaler.fit_transform(train_test.iloc[:,1:])

# 특성 차원이 너무 많을 경우 과적합이 발생하기 때문에 차원 축소를 실행한다.
max_d = num_d = train_test.shape[1] - 1
pca = PCA(n_components=max_d, random_state=0).fit(train_test.iloc[:,1:])
cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합
num_d = np.argmax(cumsum >= 0.99) + 1             # 분산의 설명량이 99%이상 되는 차원의 수
if num_d == 1: num_d = max_d
pca = PCA(n_components=num_d, random_state=0).fit_transform(train_test.iloc[:,1:])
train_test = pd.concat([train_test.iloc[:,0], pd.DataFrame(pca)], axis=1)
display(train_test)

# 전처리 후 학습용과 제출용 데이터로 분리한다.
X_train = train_test.query('cust_id not in @IDtest').drop('cust_id', axis=1)
X_test = train_test.query('cust_id in @IDtest').drop('cust_id', axis=1)


seed = 2020
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

i = int(round(X_train.shape[0] * 0.8, 0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

,cust_id,0,1,2,3,4,5,6,7,8,...,100,101,102,103,104,105,106,107,108,109
0,0,0.625380,0.034396,-0.672246,0.441779,0.159294,-0.598718,0.325886,-0.936302,0.305429,...,-0.143142,0.118564,-0.002334,0.089901,-0.111054,-0.021497,0.140365,0.117912,-0.126485,0.046712
1,1,-1.107635,-0.008147,-0.036582,-0.227227,-0.173067,0.257546,0.036107,-0.059078,-0.073846,...,0.030561,-0.018014,0.032755,0.015114,0.011464,-0.002095,-0.073120,-0.051948,0.005610,-0.007939
2,2,-1.099033,-0.109059,-0.020829,0.007957,-0.059710,0.090729,0.179059,-0.068382,0.062943,...,0.002262,0.035945,-0.017813,-0.008323,-0.036238,0.021623,0.038809,0.046284,0.007039,0.014445
3,3,0.265591,-0.619277,-0.279149,-0.187955,0.023135,-0.135279,-0.067050,0.675716,-0.159730,...,0.131376,0.064030,-0.118500,0.066814,-0.149491,-0.003301,0.486243,-0.482798,-0.028555,-0.156546
4,4,-1.141518,-0.191579,0.005533,-0.059629,0.026487,0.013643,-0.095080,0.043151,-0.057431,...,0.008822,0.004029,0.004533,-0.000862,-0.005974,-0.006765,0.007707,0.013415,-0.002152,0.008844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5977,5977,-0.877498,0.236507,-0.127787,-0.066931,-0.135133,0.047662,0.136196,-0.051518,-0.006189,...,0.100014,-0.477439,-0.028659,0.204758,-0.015639,0.083038,0.010453,-0.005576,-0.006840,0.081172
5978,5978,-1.162111,-0.159106,0.019576,0.078777,0.026611,-0.007421,-0.020261,0.025530,-0.021468,...,0.005455,0.005500,0.013200,0.006884,0.011396,0.006313,-0.050464,-0.032066,-0.036101,-0.007855
5979,5979,0.199450,1.781264,-0.540263,0.071173,0.331206,1.250888,0.351207,-0.141133,-0.117322,...,0.100453,0.028486,-0.128008,0.088439,-0.112445,-0.079405,0.109667,0.038992,-0.088774,0.054421
5980,5980,-0.635700,-0.227858,-0.237216,0.009492,-0.166560,-0.074526,0.129139,-0.274902,-0.380003,...,-0.285796,0.651523,0.113559,-0.241631,0.065565,0.000818,-0.040256,-0.085648,-0.044685,-0.053650


In [4]:
   
# model architecture
model = Sequential(name = 'dnn model')
model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# choose the optimizer and the cost function
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# model.summary()


# train the model -> verbose=0: silent
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)]

hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
    batch_size=64, epochs=200, callbacks=callbacks, verbose=0)

# visualize training history
# plt.plot(hist.history['loss'], label='train loss')
# plt.plot(hist.history['val_loss'], label='validation loss')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('Loss')
# plt.show()


# plt.plot(hist.history['acc'], label='train acc')
# plt.plot(hist.history['val_acc'], label='validation acc')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('acc')
# plt.show()

# evaluate the model performance

#print(model.evaluate(X_test, y_test))
#if roc_auc_score(y_test, model.predict(X_test)) >= 0.755:
#    print(f'1층: {dr1},드롭1: {drop1},2층: {dr2}, 드롭2: {drop2}, 3층: {dr3}')
#    print(roc_auc_score(y_test, model.predict(X_test)))
print(roc_auc_score(y_valid, model.predict(X_valid)))

0.7325722824576035


# 대분류 구매건수

In [28]:
df_train = pd.read_csv('X_train.csv', encoding='cp949')
df_test = pd.read_csv('X_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949').gender
IDtest = df_test.cust_id.unique()


level = 'gds_grp_mclas_nm'

train_test = pd.pivot_table(pd.concat([df_train, df_test]), index='cust_id', columns=level, values='amount',
                            aggfunc=lambda x: len(x), fill_value=0).reset_index()


# 이상치(outlier)를 제거한다.
train_test.iloc[:,1:] = train_test.iloc[:,1:].apply(lambda x: x.clip(x.quantile(.05), x.quantile(.95)), axis=0)

# 왼쪽으로 치우진 분포를 정규분포로 바꾸기 위해 로그 변환을 수행한다. -> 0.769
train_test.iloc[:,1:] = np.log1p(train_test.iloc[:,1:])

#from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler()
train_test.iloc[:, 1:] = mmscaler.fit_transform(train_test.iloc[:,1:])

# 특성 차원이 너무 많을 경우 과적합이 발생하기 때문에 차원 축소를 실행한다.
max_d = num_d = train_test.shape[1] - 1
pca = PCA(n_components=max_d, random_state=0).fit(train_test.iloc[:,1:])
cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합
num_d = np.argmax(cumsum >= 0.99) + 1             # 분산의 설명량이 99%이상 되는 차원의 수
if num_d == 1: num_d = max_d
pca = PCA(n_components=num_d, random_state=0).fit_transform(train_test.iloc[:,1:])
train_test = pd.concat([train_test.iloc[:,0], pd.DataFrame(pca)], axis=1)
display(train_test)

# 전처리 후 학습용과 제출용 데이터로 분리한다.
X_train = train_test.query('cust_id not in @IDtest').drop('cust_id', axis=1)
X_test = train_test.query('cust_id in @IDtest').drop('cust_id', axis=1)


seed = 2020
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

i = int(round(X_train.shape[0] * 0.8, 0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

,cust_id,0,1,2,3,4,5,6,7,8,...,25,26,27,28,29,30,31,32,33,34
0,0,0.808812,0.047621,-0.382530,0.136842,0.068270,-1.067826,0.097073,0.064659,0.123847,...,0.278258,-0.198159,-0.226099,-0.038080,0.324296,-0.434571,-0.308631,-0.016363,0.194200,-0.018975
1,1,-0.942326,0.181016,0.039485,0.224548,0.165447,0.185474,0.260437,0.067577,-0.233817,...,-0.035076,-0.147834,0.062239,0.060819,-0.054463,-0.387346,0.112056,-0.059185,0.006323,-0.030976
2,2,-0.939487,0.072728,-0.171136,0.257513,-0.163384,-0.015842,-0.109646,0.098185,-0.189282,...,0.057341,0.047966,-0.038936,-0.112071,-0.006490,0.062245,-0.004990,0.035366,0.044459,0.039179
3,3,0.176414,-0.540114,-0.095850,0.039596,0.054163,0.377580,0.169790,-0.019942,0.183439,...,0.156863,-0.014416,-0.061704,0.120444,-0.374014,0.177117,-0.243579,0.075744,-0.142311,-0.229687
4,4,-1.006475,-0.015659,0.083409,-0.036429,0.082568,-0.020770,0.012661,-0.029358,0.030033,...,0.008659,0.040425,-0.007492,0.042274,0.049531,-0.034650,-0.017044,-0.018064,0.021210,-0.000362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5977,5977,-0.527805,0.460797,-0.064967,-0.029257,0.232430,-0.141500,0.104384,0.065651,-0.706434,...,-0.477599,0.215720,0.023805,0.367133,0.026163,0.424302,-0.089105,-0.048746,0.062714,-0.055811
5978,5978,-1.038196,0.022598,0.023371,-0.003908,0.008111,0.030509,-0.068062,-0.030968,0.011029,...,0.106910,0.017061,0.158339,-0.014808,-0.045128,0.014905,-0.012643,-0.058065,0.033216,-0.046763
5979,5979,0.672151,1.288926,0.047105,0.442555,-0.225962,0.620944,-0.542938,-0.306575,-0.724191,...,-0.267473,0.172990,0.246498,-0.123809,-0.203993,-0.005225,-0.029028,0.124641,0.155520,0.100988
5980,5980,-0.561052,-0.015907,0.100263,0.082922,-0.023670,0.353346,0.424770,-0.039177,0.215357,...,0.149904,-0.020741,-0.158265,0.161862,-0.201403,-0.103223,-0.567873,-0.410687,-0.080075,0.002968


In [29]:
   
# model architecture
model = Sequential(name = 'dnn model')
model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# choose the optimizer and the cost function
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# model.summary()


# train the model -> verbose=0: silent
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)]

hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
    batch_size=64, epochs=200, callbacks=callbacks, verbose=0)

# visualize training history
# plt.plot(hist.history['loss'], label='train loss')
# plt.plot(hist.history['val_loss'], label='validation loss')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('Loss')
# plt.show()


# plt.plot(hist.history['acc'], label='train acc')
# plt.plot(hist.history['val_acc'], label='validation acc')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('acc')
# plt.show()

# evaluate the model performance

#print(model.evaluate(X_test, y_test))
#if roc_auc_score(y_test, model.predict(X_test)) >= 0.755:
#    print(f'1층: {dr1},드롭1: {drop1},2층: {dr2}, 드롭2: {drop2}, 3층: {dr3}')
#    print(roc_auc_score(y_test, model.predict(X_test)))
print(roc_auc_score(y_valid, model.predict(X_valid)))

0.7503127606338615


# 중분류 구매여부

In [42]:
df_train = pd.read_csv('X_train.csv', encoding='cp949')
df_test = pd.read_csv('X_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949').gender
IDtest = df_test.cust_id.unique()


level = 'gds_grp_nm'

train_test = pd.pivot_table(pd.concat([df_train, df_test]), index='cust_id', columns=level, values='amount',
                           aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0).reset_index()


# 이상치(outlier)를 제거한다.
train_test.iloc[:,1:] = train_test.iloc[:,1:].apply(lambda x: x.clip(x.quantile(.05), x.quantile(.95)), axis=0)

# 왼쪽으로 치우진 분포를 정규분포로 바꾸기 위해 로그 변환을 수행한다. -> 0.769
train_test.iloc[:,1:] = np.log1p(train_test.iloc[:,1:])

#from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler()
train_test.iloc[:, 1:] = mmscaler.fit_transform(train_test.iloc[:,1:])

# 특성 차원이 너무 많을 경우 과적합이 발생하기 때문에 차원 축소를 실행한다.
max_d = num_d = train_test.shape[1] - 1
pca = PCA(n_components=max_d, random_state=0).fit(train_test.iloc[:,1:])
cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합
num_d = np.argmax(cumsum >= 0.99) + 1             # 분산의 설명량이 99%이상 되는 차원의 수
if num_d == 1: num_d = max_d
pca = PCA(n_components=num_d, random_state=0).fit_transform(train_test.iloc[:,1:])
train_test = pd.concat([train_test.iloc[:,0], pd.DataFrame(pca)], axis=1)
display(train_test)

# 전처리 후 학습용과 제출용 데이터로 분리한다.
X_train = train_test.query('cust_id not in @IDtest').drop('cust_id', axis=1)
X_test = train_test.query('cust_id in @IDtest').drop('cust_id', axis=1)


seed = 2020
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

i = int(round(X_train.shape[0] * 0.8, 0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

,cust_id,0,1,2,3,4,5,6,7,8,...,101,102,103,104,105,106,107,108,109,110
0,0,1.031835,0.212301,-0.922307,-0.869803,-0.305008,0.092058,0.831769,-0.575745,-0.703853,...,0.102421,-0.318619,-0.375416,-0.050509,0.185363,-0.019726,-0.122237,-0.086713,-0.053711,-0.077012
1,1,-1.545397,-0.120407,0.028675,0.202452,0.264835,0.240070,0.129344,0.058812,0.209729,...,0.028266,0.074902,0.029886,0.116798,-0.031030,-0.049614,0.024517,0.003476,0.001157,0.006372
2,2,-1.415123,-0.065534,0.005330,-0.016653,0.142608,0.400511,-0.053878,-0.125965,-0.270389,...,0.018800,0.036842,-0.011082,-0.020539,-0.010853,-0.012236,-0.003550,-0.000766,0.007166,0.015099
3,3,0.865498,-1.222110,-0.553205,0.022980,0.309707,-0.115284,-0.423851,0.164560,0.401825,...,0.122099,0.050478,0.119724,-0.141897,-0.014390,0.095265,-0.023132,0.142257,0.325156,0.171970
4,4,-1.341069,-0.078229,-0.045112,0.219325,-0.030194,-0.500378,0.017921,0.145940,-0.016312,...,0.012135,0.031032,-0.017324,-0.006011,0.007325,-0.018732,0.004144,-0.010431,-0.004584,-0.006360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5977,5977,-1.161452,0.308622,-0.139144,-0.117040,0.072620,0.152300,-0.058142,-0.208458,-0.120671,...,0.004387,-0.135064,0.031170,-0.002395,-0.009385,0.033274,0.079296,0.230744,-0.209229,-0.247969
5978,5978,-1.547982,-0.166368,0.062790,-0.058637,-0.039730,0.145217,0.182716,0.213168,0.241474,...,-0.017176,-0.011808,-0.000613,-0.003491,-0.025970,-0.001241,0.008654,-0.008849,-0.015761,-0.011166
5979,5979,0.204833,1.663809,-0.728419,0.605944,0.811258,1.091609,-0.392112,1.131950,-0.592724,...,-0.523407,0.207549,-0.355611,-0.084467,0.018351,0.011924,0.144681,0.012927,0.108045,-0.044070
5980,5980,-0.925491,-0.298534,-0.188338,-0.024190,0.231828,0.287504,0.755301,-0.127350,0.329944,...,-0.028420,-0.098941,-0.130566,0.075247,-0.050180,0.088172,-0.168739,-0.393781,0.228118,0.392946


In [43]:
   
# model architecture
model = Sequential(name = 'dnn model')
model.add(Dense(16, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# choose the optimizer and the cost function
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# model.summary()


# train the model -> verbose=0: silent
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)]

hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
    batch_size=64, epochs=200, callbacks=callbacks, verbose=0)

# visualize training history
# plt.plot(hist.history['loss'], label='train loss')
# plt.plot(hist.history['val_loss'], label='validation loss')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('Loss')
# plt.show()


# plt.plot(hist.history['acc'], label='train acc')
# plt.plot(hist.history['val_acc'], label='validation acc')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('acc')
# plt.show()

# evaluate the model performance

#print(model.evaluate(X_test, y_test))
#if roc_auc_score(y_test, model.predict(X_test)) >= 0.755:
#    print(f'1층: {dr1},드롭1: {drop1},2층: {dr2}, 드롭2: {drop2}, 3층: {dr3}')
#    print(roc_auc_score(y_test, model.predict(X_test)))
print(roc_auc_score(y_valid, model.predict(X_valid)))

0.7306479357798165


# 대분류 구매여부

In [44]:
df_train = pd.read_csv('X_train.csv', encoding='cp949')
df_test = pd.read_csv('X_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949').gender
IDtest = df_test.cust_id.unique()


level = 'gds_grp_mclas_nm'

train_test = pd.pivot_table(pd.concat([df_train, df_test]), index='cust_id', columns=level, values='amount',
                           aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0).reset_index()


# 이상치(outlier)를 제거한다.
train_test.iloc[:,1:] = train_test.iloc[:,1:].apply(lambda x: x.clip(x.quantile(.05), x.quantile(.95)), axis=0)

# 왼쪽으로 치우진 분포를 정규분포로 바꾸기 위해 로그 변환을 수행한다. -> 0.769
train_test.iloc[:,1:] = np.log1p(train_test.iloc[:,1:])

#from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler()
train_test.iloc[:, 1:] = mmscaler.fit_transform(train_test.iloc[:,1:])

# 특성 차원이 너무 많을 경우 과적합이 발생하기 때문에 차원 축소를 실행한다.
max_d = num_d = train_test.shape[1] - 1
pca = PCA(n_components=max_d, random_state=0).fit(train_test.iloc[:,1:])
cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합
num_d = np.argmax(cumsum >= 0.99) + 1             # 분산의 설명량이 99%이상 되는 차원의 수
if num_d == 1: num_d = max_d
pca = PCA(n_components=num_d, random_state=0).fit_transform(train_test.iloc[:,1:])
train_test = pd.concat([train_test.iloc[:,0], pd.DataFrame(pca)], axis=1)
display(train_test)

# 전처리 후 학습용과 제출용 데이터로 분리한다.
X_train = train_test.query('cust_id not in @IDtest').drop('cust_id', axis=1)
X_test = train_test.query('cust_id in @IDtest').drop('cust_id', axis=1)


seed = 2020
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

i = int(round(X_train.shape[0] * 0.8, 0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

,cust_id,0,1,2,3,4,5,6,7,8,...,26,27,28,29,30,31,32,33,34,35
0,0,1.436058,-0.040728,-0.376852,0.244498,-0.337669,0.594535,1.160821,-0.507886,-0.026341,...,0.371315,-0.820841,0.280656,0.359050,-0.052447,0.167750,-0.028469,0.045124,0.103598,0.014635
1,1,-1.495079,0.426246,0.603980,0.267780,0.167863,-0.341812,0.074077,0.082130,-0.268278,...,-0.027723,-0.408299,-0.796369,-0.191770,0.136208,0.175161,-0.027328,0.023678,0.019509,-0.013586
2,2,-1.486196,0.137751,0.094870,0.455594,0.025952,0.268160,0.156926,-0.156983,-0.082030,...,0.102234,0.202870,0.137714,0.092741,-0.046748,-0.249884,0.133006,0.062282,-0.045456,-0.040493
3,3,0.826307,-0.968225,0.253546,0.553680,-0.338130,-0.531447,-0.663852,0.274229,0.311605,...,-0.615561,0.226854,0.448593,0.001418,-0.397996,-0.505601,-0.206288,-0.023323,-0.058132,-0.078507
4,4,-1.388948,0.099101,0.054519,-0.485316,-0.493745,0.040495,0.094735,-0.274916,-0.333733,...,0.094130,0.096581,-0.122793,0.078391,-0.059339,-0.210875,0.119112,0.095087,-0.033616,0.010124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5977,5977,-0.758481,0.812565,-0.108292,0.172876,0.196507,-0.143411,0.183231,0.159825,-0.234539,...,-0.780837,-0.074323,-0.534725,-0.266745,-0.442471,-0.236793,-0.081901,0.159949,-0.030645,0.010190
5978,5978,-1.564872,0.227113,0.097719,0.005096,-0.014250,0.254983,-0.491875,-0.342264,0.369359,...,-0.097276,-0.100308,0.012817,0.030391,0.023927,0.100438,-0.032773,-0.034280,-0.015645,0.006128
5979,5979,1.010739,1.482501,1.119399,0.718032,-0.021002,0.340607,-0.337983,0.478843,0.405145,...,0.111127,0.135636,-0.055767,-0.729023,-0.262131,0.856770,0.272822,0.424178,-0.637153,-0.439690
5980,5980,-0.907570,0.014547,0.360717,0.103525,0.415867,-0.849732,0.382268,-0.345673,0.521682,...,-0.653179,0.014776,0.284553,-0.030792,-0.207601,-0.053542,-0.043660,-0.118073,-0.046550,-0.100628


In [45]:
   
# model architecture
model = Sequential(name = 'dnn model')
model.add(Dense(16, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# choose the optimizer and the cost function
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# model.summary()


# train the model -> verbose=0: silent
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)]

hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
    batch_size=64, epochs=200, callbacks=callbacks, verbose=0)

# visualize training history
# plt.plot(hist.history['loss'], label='train loss')
# plt.plot(hist.history['val_loss'], label='validation loss')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('Loss')
# plt.show()


# plt.plot(hist.history['acc'], label='train acc')
# plt.plot(hist.history['val_acc'], label='validation acc')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('acc')
# plt.show()

# evaluate the model performance

#print(model.evaluate(X_test, y_test))
#if roc_auc_score(y_test, model.predict(X_test)) >= 0.755:
#    print(f'1층: {dr1},드롭1: {drop1},2층: {dr2}, 드롭2: {drop2}, 3층: {dr3}')
#    print(roc_auc_score(y_test, model.predict(X_test)))
print(roc_auc_score(y_valid, model.predict(X_valid)))

0.7421766402557687


# 중분류 구매건수 percentile

In [54]:
X_train = pd.read_csv('train_numbersOfPurchase_nm_percentile.csv', encoding='cp949')
X_test = pd.read_csv('train_numbersOfPurchase_nm_percentile.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949').gender

seed = 2020
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

i = int(round(X_train.shape[0] * 0.8, 0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

In [55]:
   
# model architecture
model = Sequential(name = 'dnn model')
model.add(Dense(16, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# choose the optimizer and the cost function
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# model.summary()


# train the model -> verbose=0: silent
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)]

hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
    batch_size=64, epochs=200, callbacks=callbacks, verbose=0)

# visualize training history
# plt.plot(hist.history['loss'], label='train loss')
# plt.plot(hist.history['val_loss'], label='validation loss')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('Loss')
# plt.show()


# plt.plot(hist.history['acc'], label='train acc')
# plt.plot(hist.history['val_acc'], label='validation acc')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('acc')
# plt.show()

# evaluate the model performance

#print(model.evaluate(X_test, y_test))
#if roc_auc_score(y_test, model.predict(X_test)) >= 0.755:
#    print(f'1층: {dr1},드롭1: {drop1},2층: {dr2}, 드롭2: {drop2}, 3층: {dr3}')
#    print(roc_auc_score(y_test, model.predict(X_test)))
print(roc_auc_score(y_valid, model.predict(X_valid)))

0.7577668890742285


# All Bows - indiviual_task1

# All Bows - group_task1

In [2]:
# X_train, X_test, y_train, y_test, X_dep, ID_dep = pd.read_pickle('case3_train_test.pkl')
# X_train.shape, X_test.shape


seed = 2020
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

i = int(round(X_train.shape[0] * 0.8, 0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

model architecture

### test other features basic scores

In [20]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,109
0,0.625380,0.034396,-0.672246,0.441779,0.159294,-0.598718,0.325886,-0.936302,0.305429,-0.529067,...,-0.143142,0.118564,-0.002334,0.089901,-0.111054,-0.021497,0.140365,0.117912,-0.126485,0.046712
1,-1.107635,-0.008147,-0.036582,-0.227227,-0.173067,0.257546,0.036107,-0.059078,-0.073846,-0.007021,...,0.030561,-0.018014,0.032755,0.015114,0.011464,-0.002095,-0.073120,-0.051948,0.005610,-0.007939
2,-1.099033,-0.109059,-0.020829,0.007957,-0.059710,0.090729,0.179059,-0.068382,0.062943,-0.019888,...,0.002262,0.035945,-0.017813,-0.008323,-0.036238,0.021623,0.038809,0.046284,0.007039,0.014445
3,0.265591,-0.619277,-0.279149,-0.187955,0.023135,-0.135279,-0.067050,0.675716,-0.159730,-0.068754,...,0.131376,0.064030,-0.118500,0.066814,-0.149491,-0.003301,0.486243,-0.482798,-0.028555,-0.156546
4,-1.141518,-0.191579,0.005533,-0.059629,0.026487,0.013643,-0.095080,0.043151,-0.057431,-0.002027,...,0.008822,0.004029,0.004533,-0.000862,-0.005974,-0.006765,0.007707,0.013415,-0.002152,0.008844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,-1.046915,-0.140033,0.103378,0.049180,0.117287,-0.135593,0.194944,0.064718,-0.104468,0.140249,...,-0.018055,0.042107,0.073418,0.032542,-0.048577,0.024386,-0.007156,-0.040956,0.002725,-0.006824
2796,1.527293,1.673800,0.590022,-0.410030,0.012423,-0.644904,-0.296354,-0.389029,0.611323,-0.646424,...,-0.118156,-0.154492,0.027863,0.238299,0.782080,-0.166157,0.164577,-0.059302,-0.306662,0.037948
2797,-1.000736,0.061578,-0.065981,-0.128722,-0.158024,0.071688,-0.036240,0.063314,-0.170748,0.247493,...,0.063057,0.029071,-0.046326,-0.022041,-0.008329,-0.034907,-0.037619,-0.025569,-0.019030,-0.012227
2798,-0.940100,-0.410004,-0.085304,0.097139,-0.015143,-0.038927,0.177830,-0.246140,-0.151301,-0.015606,...,0.006842,0.033337,0.072775,0.052616,0.017694,0.026172,-0.053506,-0.042600,-0.012527,-0.031088


In [25]:
   
# model architecture
model = Sequential(name = 'dnn model')
model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# choose the optimizer and the cost function
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# model.summary()


# train the model -> verbose=0: silent
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)]

hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
    batch_size=64, epochs=200, callbacks=callbacks, verbose=0)

# visualize training history
# plt.plot(hist.history['loss'], label='train loss')
# plt.plot(hist.history['val_loss'], label='validation loss')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('Loss')
# plt.show()


# plt.plot(hist.history['acc'], label='train acc')
# plt.plot(hist.history['val_acc'], label='validation acc')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('acc')
# plt.show()

# evaluate the model performance

#print(model.evaluate(X_test, y_test))
#if roc_auc_score(y_test, model.predict(X_test)) >= 0.755:
#    print(f'1층: {dr1},드롭1: {drop1},2층: {dr2}, 드롭2: {drop2}, 3층: {dr3}')
#    print(roc_auc_score(y_test, model.predict(X_test)))
print(roc_auc_score(y_valid, model.predict(X_valid)))

0.7280372532666111


In [139]:
dr1st = dense_range_1st_layer = [4, 8, 16, 32]
dr2nd = dense_range_2nd_layer = [4, 8, 16, 32]
dr3rd = dense_range_3rd_layer = [4, 8, 16, 32]
drop1st = dropout_range_1st_layer = [0, 0.2, 0.3, 0.4, 0.5]
drop2nd = dropout_range_2nd_layer = [0, 0.2, 0.3, 0.4, 0.5]

for dr1 in dr1st:
    
    for drop1 in drop1st:
    
        for dr2 in dr2nd:
        
            for drop2 in drop2nd:
                
                for dr3 in dr3rd:

        
# model architecture
                    model = Sequential(name = 'dnn model')
                    model.add(Dense(dr1, input_shape=(X_train.shape[1],), activation='relu'))
                    model.add(Dropout(drop1))
                    model.add(Dense(dr2, activation='relu'))
                    model.add(Dropout(drop2))
                    model.add(Dense(dr3, activation='relu'))
                    model.add(Dense(1, activation='sigmoid'))

# choose the optimizer and the cost function
                    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# model.summary()


# train the model -> verbose=0: silent
                    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)]

                    hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                        batch_size=64, epochs=200, callbacks=callbacks, verbose=0)

# visualize training history
# plt.plot(hist.history['loss'], label='train loss')
# plt.plot(hist.history['val_loss'], label='validation loss')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('Loss')
# plt.show()


# plt.plot(hist.history['acc'], label='train acc')
# plt.plot(hist.history['val_acc'], label='validation acc')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('acc')
# plt.show()

# evaluate the model performance

#                    print(model.evaluate(X_test, y_test))
                    if roc_auc_score(y_test, model.predict(X_test)) >= 0.755:
                        print(f'1층: {dr1},드롭1: {drop1},2층: {dr2}, 드롭2: {drop2}, 3층: {dr3}')
                        print(roc_auc_score(y_test, model.predict(X_test)))

1층: 4,드롭1: 0,2층: 4, 드롭2: 0, 3층: 4
0.754451755845315
1층: 4,드롭1: 0,2층: 4, 드롭2: 0, 3층: 16
0.7520051984451639
1층: 4,드롭1: 0,2층: 8, 드롭2: 0.2, 3층: 8
0.7528840018632981
1층: 4,드롭1: 0,2층: 8, 드롭2: 0.3, 3층: 4
0.7529681634378633
1층: 4,드롭1: 0,2층: 8, 드롭2: 0.4, 3층: 8
0.7552542266725645
1층: 4,드롭1: 0,2층: 16, 드롭2: 0, 3층: 4
0.7533791850810887
1층: 4,드롭1: 0,2층: 16, 드롭2: 0.2, 3층: 8
0.7549488963090256
1층: 4,드롭1: 0,2층: 16, 드롭2: 0.3, 3층: 16
0.751797730377631
1층: 4,드롭1: 0,2층: 16, 드롭2: 0.4, 3층: 8
0.7521226332003712
1층: 4,드롭1: 0,2층: 32, 드롭2: 0, 3층: 4
0.7580433021087367
1층: 4,드롭1: 0,2층: 32, 드롭2: 0, 3층: 32
0.7507114588919639
1층: 4,드롭1: 0,2층: 32, 드롭2: 0.2, 3층: 4
0.7601042820626239
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0, 3층: 4
0.7557631106117959
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0, 3층: 8
0.7504276582335464
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0, 3층: 16
0.7534555176719734
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0, 3층: 32
0.7549919557192684
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0.3, 3층: 8
0.7572075581008451
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0.3, 3층: 16
0.7511988131260741
1층: 4

KeyboardInterrupt: 

predict unseen data

In [75]:
pred = model.predict(X_dep).flatten()
print(pred.shape)


(2482,)


In [77]:
# make submissions

pd.DataFrame({'cust_id': ID_dep, 'gender': pred}).to_csv('submission_dnn_nm.csv', index=False, encoding='cp949')
pd.read_csv('submission_dnn_nm.csv', encoding='cp949')

,cust_id,gender
0,3500,0.778295
1,3501,0.210400
2,3502,0.139749
3,3503,0.169812
4,3504,0.267438
...,...,...
2477,5977,0.516476
2478,5978,0.382308
2479,5979,0.816023
2480,5980,0.276791


save the model for future use

In [55]:
# pickle 형식으로 저장
with open('nm_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [56]:
# 추후 저장한 모형 불러올 때
model = pd.read_pickle('nm_model.pkl')

In [58]:
# h5 형식으로 저장
model.save('nm_model.h5')

In [59]:
# 추후 저장한 모형 불러올 때
model = load_model('nm_model.h5')

1. libraries, read data     
2. data engineering   
3. model architecture, choose the optimizer and the cost function, train the model   
4. visualize training history, evaluate the model performance   
5. predict unseen data, save the model for future use   